In [ ]:
import numpy as np
from scipy.integrate import simps
from joblib import Parallel, delayed
from itertools import product

In [ ]:
###############################################################################
#### Spectra generator varying time, Δm²₂₁, Δm²₃₁, θ₁₂, and θ₁₃ ###############
###############################################################################

# --- Generates the 'specs_NO.npy', 'specs_IO.npy', and 'parameters.npy' files --- #

# Computes part of the unoscillated reactor neutrinos flux at energy Enu (arXiv:1101.2663, arXiv:1411.6475).
def flux(Enu):
    
    # Isotope fractions and mean energies
    tf_iso = [0.58, 0.07, 0.3, 0.05] # U235, U238, Pu239, and Pu241
    ef_iso = [202.36, 205.99, 211.12, 214.26]
    
    # Parametrization coefficients
    a0 = [3.217, 4.833e-1, 6.413, 3.251]
    a1 = [-3.111, 1.927e-1, -7.432, -3.204]
    a2 = [1.395, -1.283e-1, 3.535, -1.428]
    a3 = [-3.69e-1, -6.762e-3, -8.82e-1, -3.675e-1]
    a4 = [4.445e-2, 2.233e-3, 1.025e-1, 4.254e-2]
    a5 = [-2.053e-3, -1.536e-4, -4.55e-3, -1.896e-3]
    
    # Calculating
    fator = 0.0
    fluxo = 0.0
    for i in range(4):
        fator += tf_iso[i]*ef_iso[i]
        fluxo += tf_iso[i] * np.exp(a0[i] + a1[i]*Enu + a2[i]*Enu**2.0 + a3[i]*Enu**3.0 + a4[i]*Enu**4.0 + a5[i]*Enu**5.0)
    
    fluxo = fluxo/fator
    return fluxo

In [ ]:
# Computes the 3-flavor electron antineutrino survival probability P_surv including constant-density matter effects.
def Psurv_withme(Enu, L, dm21, dm31, t12, t13, t23, Ye, rho):
    
    # Converting mixing angles to radians
    t12 = np.deg2rad(t12)
    t13 = np.deg2rad(t13)
    t23 = np.deg2rad(t23)
    
    # Defining Δm²₃₂
    dm32 = dm31 - dm21
    
    # Sines and cosines of the mixing angles 
    s2_t12 = np.sin(t12)**2
    c2_t12 = np.cos(t12)**2
    c_2t12 = np.cos(2.0*t12)
    s2_2t12 = np.sin(2.0*t12)**2
    s2_2t13 = np.sin(2.0*t13)**2
    c4_t13 = np.cos(t13)**4
    
    # Oscillation phases in vacuum
    d21 = 1.27e3*dm21*L/Enu
    d31 = 1.27e3*dm31*L/Enu
    d32 = 1.27e3*dm32*L/Enu
    s2_d21 = np.sin(d21)**2
    s2_d31 = np.sin(d31)**2
    s2_d32 = np.sin(d32)**2
    
    # Matter potential (constant density approximation)
    Acc = 1.52e-7*Ye*rho*Enu
    
    # Mixing angle θ₁₂ and Δm²₂₁ in matter (approximate)
    s2_t12m = s2_t12*(1.0 + 2.0*Acc*np.cos(t12)/dm21)
    c2_t12m = 1 - s2_t12m
    s2_2t12m = s2_2t12*(1.0 + 2.0*Acc*c_2t12/dm21)
    dm21m = dm21*(1 - Acc*c_2t12/dm21)
    d21m = 1.27e3*dm21m*L/Enu
    s2_d21m = np.sin(d21m)**2
    
    # Final expression for P_surv with matter effects
    prob = 1.0 - s2_2t12m*c4_t13*s2_d21m - s2_2t13*(c2_t12m*s2_d31 + s2_t12m*s2_d32)
    return prob

# Computes the JUNO-averaged electron antineutrino survival probability (Normal Ordering).
def Psurv_JUNO_NO(Enu, dm21, dm31_NO, t12, t13_NO):
    
    # Reactor baselines to JUNO's main detector (L) in km
    L = [52.77, 52.77, 52.74, 52.82, 52.41, 52.49, 52.11, 52.19]
    
    # Reactor thermal powers in GWth
    W = [4.6, 4.6, 2.9, 2.9, 2.9, 2.9, 2.9, 2.9]
    
    # Fixed parameters
    t23_NO = 48.5 # θ₂₃ best-fit value for the Normal Ordering [degrees] (arXiv:2410.05380)
    Ye = 0.5 # Electron fraction of the medium
    rho = 2.45 # Earth's typical matter density [g/cm³]
    
    # Arrays to store weights and individual survival probabilities
    fator = []
    prob_NO = []
    
    # Loop over all reactors
    for i in range(len(L)):
        # Geometrical + power weight
        fator.append(W[i]/(4*np.pi*L[i]**2))
        
        # Computing P_surv for this reactor
        prob_NO.append(Psurv_withme(Enu, L[i], dm21, dm31_NO, t12, t13_NO, t23_NO, Ye, rho))
    
    # Converting to array and applying unit conversion factor
    fator = np.array(fator)*6.242e11
    prob_NO = np.array(prob_NO)
    
    # Computing the weighted average survival probability
    prob_NO = sum(fator*prob_NO)
    
    return prob_NO

# Computes the JUNO-averaged electron antineutrino survival probability (Inverted Ordering).
def Psurv_JUNO_IO(Enu, dm21, dm31_IO, t12, t13_IO):
    
    # Reactor baselines to JUNO's main detector (L) in km
    L = [52.77, 52.77, 52.74, 52.82, 52.41, 52.49, 52.11, 52.19]
    
    # Reactor thermal powers in GWth
    W = [4.6, 4.6, 2.9, 2.9, 2.9, 2.9, 2.9, 2.9]
    
    # Fixed parameters
    t23_IO = 48.6 # θ₂₃ best-fit value for the Normal Ordering [degrees] (arXiv:2410.05380)
    Ye = 0.5 # Electron fraction of the medium
    rho = 2.45 # Earth's typical matter density [g/cm³]
    
    # Arrays to store weights and individual survival probabilities
    fator = []
    prob_IO = []
    
    # Loop over all reactors
    for i in range(len(L)):
        # Geometrical + power weight
        fator.append(W[i]/(4*np.pi*L[i]**2))
        
        # Computing P_surv for this reactor
        prob_IO.append(Psurv_withme(Enu, L[i], dm21, dm31_IO, t12, t13_IO, t23_IO, Ye, rho))
    
    # Converting to array and applying unit conversion factor
    fator = np.array(fator)*6.242e11
    prob_IO = np.array(prob_IO)
    
    # Computing the weighted average survival probability
    prob_IO = sum(fator*prob_IO)
    
    return prob_IO

In [ ]:
# Computes an expected observed energy spectrum at JUNO considering the Normal Ordering.
def Spectrum_NO(t, dm21, dm31_NO, t12, t13_NO):
    
    # Loading the precomputed associated antineutrino energy range and IBD effective cross-section for each bin
    enubins=np.load('junoenuspec.npy')
    sigbins=np.load('junosigspec.npy')
    
    # Loading the total background spectrum
    Bg_spectra = np.load('Bg_spectra_tot.npy')
    
    # Initializing output array
    sbins_osc_NO = np.zeros(len(enubins))
    
    # Looping over the energy bins 
    for i in range(len(enubins)):
        enn = enubins[i] # antineutrino energy range in bin i
        bg = Bg_spectra[i] # background in this bin
        spectmp_osc_NO = []
        
        # Looping over the antineutrino energies within bin i
        for j in range(len(enubins[0])):
            enu = enn[j]
            
            # Reactor flux
            fluxnu = flux(enu)
            
            # JUNO-averaged survival probability (NO)
            pmed_NO = Psurv_JUNO_NO(enu, dm21, dm31_NO, t12, t13_NO)
            
            # Multiplying flux × probability × cross-section
            spectmp_osc_NO.append(fluxnu * pmed_NO * sigbins[i][j])
            
        Np = 1.44e33 # Number of target protons in JUNO
        eff = 0.822 # Detection efficiency
        normaliz = Np*eff*t # factor including the exposure time t
        
        # Expected number of events in bin i
        sbins_osc_NO[i] = normaliz*(bg/(Np*eff) + simps(spectmp_osc_NO, enn, even='avg'))
        
    return sbins_osc_NO

# Computes an expected observed energy spectrum at JUNO considering the Inverted Ordering.
def Spectrum_IO(t, dm21, dm31_IO, t12, t13_IO):
    
    # Loading the precomputed associated antineutrino energy range and IBD effective cross-section for each bin
    enubins=np.load('junoenuspec.npy')
    sigbins=np.load('junosigspec.npy')
    
    # Loading the total background spectrum
    Bg_spectra = np.load('Bg_spectra_tot.npy')
    
    # Initializing output array
    sbins_osc_IO = np.zeros(len(enubins))
    
    # Looping over the energy bins 
    for i in range(len(enubins)):
        enn = enubins[i] # antineutrino energy range in bin i
        bg = Bg_spectra[i] # background in this bin
        spectmp_osc_IO = []
        
        # Looping over the antineutrino energies within bin i
        for j in range(len(enubins[0])):
            enu = enn[j]
            
            # Reactor flux
            fluxnu = flux(enu)
            
            # JUNO-averaged survival probability (IO)
            pmed_IO = Psurv_JUNO_IO(enu, dm21, dm31_IO, t12, t13_IO)
            
            # Multiplying flux × probability × cross-section
            spectmp_osc_IO.append(fluxnu * pmed_IO * sigbins[i][j])
            
        Np = 1.44e33 # Number of target protons in JUNO
        eff = 0.822 # Detection efficiency
        normaliz = Np*eff*t # factor including the exposure time t
        
        # Expected number of events in bin i
        sbins_osc_IO[i] = normaliz*(bg/(Np*eff) + simps(spectmp_osc_IO, enn, even='avg'))
        
    return sbins_osc_IO

In [ ]:
# A wrapper function to compute a pair of JUNO spectra (NO and IO)for a given combination of oscillation parameters and time.
def Spectra(t, i, j, k, l):
    
    # Computing the expected spectrum for Normal Ordering
    spec_NO_tmp = Spectrum_NO(t_list[t], dm21_list[i], dm31_NO_list[j], t12_list[k], t13_NO_list[l])
    
    # Computing the expected spectrum for Inverted Ordering
    spec_IO_tmp = Spectrum_IO(t_list[t], dm21_list[i], dm31_IO_list[j], t12_list[k], t13_IO_list[l])
    
    # Returns all indices and both spectra
    return (t, i, j, k, l, spec_NO_tmp, spec_IO_tmp)

In [ ]:
# How many points for each parameter grid?
# User enters "n1 n2 n3 n4"
# n1 = number of Δm²21 points
# n2 = number of Δm²31 points
# n3 = number of θ12 points
# n4 = number of θ13 points
valores = [i for i in input('Enter the number of Δm²21, Δm²31, θ12 e θ13 value ranges: ').split()]
n1,n2,n3,n4 = int(valores[0]), int(valores[1]), int(valores[2]), int(valores[3])

# Defining the time grid
t_tot = 86400*365*10 # Total exposure time [seconds] = 10 years = 86400 × 365 × 10

# How often to compute spectra?
# Example: if user inputs "6", T = 120/6 = 20 → 20 points (every 6 months)
T = 120//int(input('How often? (months): '))

# Step size in seconds between time points
inc = t_tot//T

# The time grid
t_list = np.arange(inc, t_tot + inc, inc)

# Loading the precomputed associated antineutrino energy range for each bin
enubins=np.load('junoenuspec.npy')
n5 = len(enubins) # Number of energy bins

# Initializing 6D arrays to store the spectra
specs_NO = np.zeros((T, n1+1, n2+1, n3+1, n4+1, n5+1))
specs_IO = np.zeros((T, n1+1, n2+1, n3+1, n4+1, n5+1))

# Printing array shape for confirmation
print('')
print('Specs size: ', np.shape(specs_NO))

# Oscillation parameters best-fit values (arXiv:2410.05380)
dm21_bf = 7.49e-5
dm31_NO_bf = 2.534e-3
dm31_IO_bf = -2.51e-3
t12_bf = 33.68
t13_NO_bf = 8.52
t13_IO_bf = 8.58

# Defining the parameter values grid
# ±2.5% equally spaced and centered at the best-fit for each parameter

dm21_list = np.linspace(dm21_bf*0.975, dm21_bf*1.025, n1+1)
print('Δm²21 list: ', dm21_list)

dm31_NO_list = np.linspace(dm31_NO_bf*0.975, dm31_NO_bf*1.025, n1+1)
print('Δm²31 NO list: ', dm31_NO_list)   

dm31_IO_list = np.linspace(dm31_IO_bf*1.025, dm31_IO_bf*0.975, n2+1)
print('Δm²31 IO list: ', dm31_IO_list)

t12_list = np.linspace(t12_bf*0.975, t12_bf*1.025, n3+1)
print('θ12 list: ', t12_list)

# If n4 == 0 → θ13 fixed at its best-fit value (negligible impact)
if n4 == 0:
    t13_NO_list = np.array([t13_NO_bf])
    print('θ13 NO list', t13_NO_list)
    
    t13_IO_list = np.array([t13_IO_bf])
    print('θ13 IO list', t13_IO_list)
    
else:
    t13_NO_list = np.linspace(t13_NO_bf*0.975, t13_NO_bf*1.025, n4+1)
    print('θ13 NO list', t13_NO_list)

    t13_IO_list = np.linspace(t13_IO_bf*0.975, t13_IO_bf*1.025, n4+1)
    print('θ13 IO list', t13_IO_list)

# Saving the grid
parameters = [list(t_list), list(dm21_list), list(dm31_NO_list), list(dm31_IO_list), list(t12_list), list(t13_NO_list), list(t13_IO_list)]
parameters = np.array(parameters, dtype = object)
np.save('parameters', parameters)

# Printing the total number of spectra to generated and corresponding tasks to be performed
NoE = (T)*(n1+1)*(n2+1)*(n3+1)*(n4+1)*2
print('')
print('Number of spectra to be generated:',NoE)
print('Number of tasks:',NoE//2)
print('')

# Parallelizing to speed up the code execution
if __name__ == "__main__": # Windows requirement
    index = list(product(range(len(t_list)), range(len(dm21_list)), range(len(dm31_NO_list)), range(len(t12_list)), range(len(t13_NO_list))))
    result = Parallel(n_jobs=-1, verbose=11)(delayed(Spectra)(t, i, j, k, l) for (t, i, j, k, l) in index)
    for t, i, j, k, l, spec_NO_tmp, spec_IO_tmp in result:
        for m in range(len(spec_NO_tmp)):
            # Saving results
            specs_NO[t][i][j][k][l][m] = spec_NO_tmp[m]
            specs_IO[t][i][j][k][l][m] = spec_IO_tmp[m]

# Saving the final spectra
np.save('specs_NO', specs_NO)
np.save('specs_IO', specs_IO)